# Jupyter Notebook : a Swiss knife for research and teaching


<div class="alert alert-block alert-success" align="center">
<b><i>"Project Jupyter exists to develop open-source software, open-standards, and services for interactive computing across dozens of programming languages."</i></b>
</div>

- __Notebook__ for nice presentation: useful for research and TEACHING
- Code, data, explanations and visualizations are combined in one place
- Can handle equations in LaTex style
- Handle dozens of programming languages including _Python_, _Juila_, and _R_
- _Jupyter Notebook_ is directly installed with __conda__ environment

## Example 1 : DFN modelling

A DFN (Discrete Fracture Network) is simplified discrete representation of fractured media. In $D$-dimensions, fractures are represented as objects of dimension $D-1$.

Natural fracture networks are found to follow power-law size distributions (ref):

<center>
$n(l)=\alpha.l^{-a}$
</center>

So that the total number of fractures $N$ in the system of size $L$ and dimension $D$ can be written as:

<center>
$N=  \int_{l_{min}}^{l_{max}} n(l).L^{D} \, \mathrm{d}l $
</center>

__GENERATE YOU OWN DFN, PLAY WITH PARAMTERS__


In [ ]:
%matplotlib inline
import sys
import time
import k3d
import os
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as pyplot 
import matplotlib.colors
import numpy as np
import pylab as pylab
import ipywidgets as widgets
mypath = os.getcwd()

import dfnlab.DFNBasis as dfnBasis
import dfnlab.DFNGenerator as dfnGen
import dfnlab.DFNAnalysis as dfnAnalysis
import dfnlab.DFNIO as dfnIO

def plot_dfn(dfn,namedir,name):
    """ plot a DFN within a jupyter notebook 
        as attr
    """
    import dfnlab.DFNIO as dfnIO
    import vtk
    fout = os.path.join(namedir,'dfnvtkdata_{}.vtp'.format(name))
    dfnIO.write_DFN_file(dfn,fout)
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(fout)
    reader.Update()
    model_matrix = (
         1.0,  0.0, 0.0, 0.0,
         0.0,  0.0, 1.0, 0.0,
         0.0,  1.0, 0.0, 0.0,
         0.0,  0.0, 0.0, 1.0
    )
    plot = k3d.plot()
    size = dfn.getSystem().getSize()
    dfn3d = k3d.vtk_poly_data(reader.GetOutput(), color_attribute = ('FractureSize',0,size), color_map=k3d.basic_color_maps.Jet, model_matrix=model_matrix)
    plot += dfn3d
    plot.display()

__Choose your parameters__

In [ ]:
N_widget = widgets.Text(value='50000', disabled=False, description='N = ')
display(N_widget)
lmin_widget = widgets.Text(value='1', disabled=False, description='lmin = ')
display(lmin_widget)
lmax_widget = widgets.Text(value='100', disabled=False, description='lmax = ')
display(lmax_widget)
L_widget = widgets.Text(value='100', disabled=False, description='L = ')
display(L_widget)
a_slider = widgets.FloatSlider(value=3.0,min=0.1,max=5.0,step=0.1,description='a = ')
display(a_slider)

In [ ]:
a = a_slider.value
N = int(N_widget.value)
lmin = float(lmin_widget.value)
lmax = float(lmax_widget.value)
L = float(L_widget.value)

system = dfnBasis.System()
system.buildParallelepiped([0, 0, 0],L)
fnet = dfnBasis.DFN(system)
generator = dfnGen.PoissonGenerator(fnet, 1001)
generator.setSizesPowerlaw(a,lmin,lmax)
generator.setPositionsUniform()
generator.setOrientationsUniform()
generator.setStopNumber(N)
generator.generate()
plot_dfn(fnet,mypath,'my_dfn')

__Plot the size distribution__

In [ ]:
def plot_size_distribution(size_distribution, figureName):
    csfont = {'fontname':'Times New Roman'}
    pyplot.rcParams['mathtext.fontset'] = 'dejavuserif'
    fig = pyplot.figure()
    ax = pyplot.axes()
    ax.set_xscale("log")
    ax.set_yscale("log")
    l = np.array(size_distribution['Val'])
    idx1 = [True if l[i]!=0 else False for i in range(0,len(l)) ]
    nl = np.array(size_distribution['Pdf'])
    idx2 = [True if nl[i]!=0 else False for i in range(0,len(nl)) ]
    idx = idx1 and idx2
    l = l[idx]
    nl = nl[idx]
    ax.plot(l,nl,marker='o',markersize=5, linewidth=0, color='k',label='DFN')

    log_l = np.log10(l)
    log_nl = np.log10(nl)
    idx = np.isfinite(log_l) & np.isfinite(log_nl)
    m, b = pylab.polyfit(log_l[idx],log_nl[idx],1)
    a=-m
    alpha = 10**b 
    print("alpha = {}".format(alpha))
    print("a = {}".format(a))
    fit = [alpha*pow(elt,-a) for elt in l]
    lab = '~$'+"{:.2e}".format(alpha)+'*l^{-'+str(round(a,2))+'}$'
    ax.plot(l,fit,marker='o', linestyle='dashed', markersize=0, linewidth=2, color='grey', label=lab)

    pyplot.xlabel('Fracture size $l$',**csfont)
    pyplot.ylabel('Fracture size distribution $n(l)$',**csfont)
    pyplot.legend()
    return fig
    
    
analyser = dfnAnalysis.DFNAnalyser(fnet)
nl = analyser.sizeDistribution(20); # number of bins
print("N = {}".format(fnet.nbFractures()))
print("lmin = {}".format(fnet.minFractureSize()))
print("lmax = {}".format(fnet.maxFractureSize()))
fig = plot_size_distribution(nl, r'size_distribution.png')

In [6]:
import ee
import ipyleaflet
# Initialize ee with access token
ee.Initialize()

C:\Users\Etudiant\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:

def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)


s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2018-01-01', '2018-01-02')

map1 = ipyleaflet.Map(
    center=(48.2082, 16.3779), zoom=4,
    layout={'height':'400px'}
)
map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(
        s2.mosaic().visualize(min=0, max=3000, gamma=1.5, bands= ['B4', 'B3', 'B2'])
    )
))
map1

In [4]:
lat = 45
lon = -100
Radius = 1000

roi = ee.Geometry.Point([lat, lon]).buffer(Radius)
point = ee.Geometry.Point (lon, lat)
start = ee.Date('2017-01-01')
finish = ee.Date('2018-12-01')
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")

Time_Set = ee.ImageCollection(
  L8.filterDate(start, finish)
    .filter(ee.Filter.lt('CLOUD_COVER', 2))
)
print('Number of images in Time Set: ', Time_Set.size().getInfo())

Space_Set = ee.ImageCollection(
    L8.filterBounds(ee.Geometry.Point(lon, lat))
)

print('Number of images in Space Set: ', Space_Set.size().getInfo())

Number of images in Time Set:  67183
Number of images in Space Set:  322
